In [1]:
import sys
sys.path.append('../')

import tquant as tq

# from datetime import date, timedelta
# import tensorflow as tf
# import pandas as pd
from datastore import *

In [2]:
def modifica_stringhe(lista_stringhe):
    # Lista per memorizzare le stringhe modificate
    stringhe_modificate = []
    
    # Rimozione del prefisso FRA
    prefisso = "FRA-"
    for stringa in lista_stringhe:
        if stringa.startswith(prefisso):
            stringa = stringa[len(prefisso):]
        stringhe_modificate.append(stringa)

    # Dizionario delle sostituzioni
    sostituzioni = {
        "1BD": "O/N",
        "2BD": "T/N",
        "3BD": "S/N",
        "CASH": "depo",
        "SWAP": "ois",
        "FRA": "fra",
    }

    # Lista temporanea per memorizzare le stringhe modificate ulteriormente
    stringhe_modificate_finali = []

    # Iterare su ciascuna stringa nella lista delle stringhe modificate
    for stringa in stringhe_modificate:
        for chiave, valore in sostituzioni.items():
            stringa = stringa.replace(chiave, valore)
        stringhe_modificate_finali.append(stringa)
    
    return stringhe_modificate_finali

In [3]:
import math

def entropy(data):
    label_counts = {}
    for entry in data:
        label = entry[-1]
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1
    entropy = 0.0
    for label in label_counts:
        prob = label_counts[label] / len(data)
        entropy -= prob * math.log2(prob)
    return entropy

def information_gain(data, attribute_index):
    attribute_values = {}
    for entry in data:
        attr_val = entry[attribute_index]
        if attr_val in attribute_values:
            attribute_values[attr_val].append(entry)
        else:
            attribute_values[attr_val] = [entry]
    gain = entropy(data)
    for attr_val in attribute_values:
        subset = attribute_values[attr_val]
        prob = len(subset) / len(data)
        gain -= prob * entropy(subset)
    return gain

def id3(data, attributes):
    labels = [entry[-1] for entry in data]
    if len(set(labels)) == 1:
        return labels[0]
    if len(attributes) == 0:
        return max(set(labels), key=labels.count)
    gains = [information_gain(data, i) for i in range(len(attributes))]
    best_attr_index = gains.index(max(gains))
    best_attr = attributes[best_attr_index]

    tree = {best_attr: {}}
    # del attributes[best_attr_index]

    attribute_values = set([entry[best_attr_index] for entry in data])
    for value in attribute_values:
        subset = [entry[:] for entry in data if entry[best_attr_index] == value]
        subtree = id3(subset, attributes[:])
        tree[best_attr][value] = subtree
    return tree

def classify(instance, tree, attributes):
    if isinstance(tree, str):
        return tree
    else:
        attr = list(tree.keys())[0]
        attr_value = ""
        if any(attr in key for key in instance.keys()):
            attr_value = instance[attr]
        subtree = tree[attr][attr_value]
        return classify(instance, subtree, attributes)

class CurveAssignment:
    def __init__(self, data, attributes):
        self.tree = id3(data, attributes)
        self.attributes = attributes

    def get_curve_name(self, instance):
        return classify(instance, self.tree, self.attributes)



In [17]:
tq.Settings.evaluation_date = date(2024, 4, 30)

deposit_builder = tq.DepositHelper(tq.Currency.EUR,
                    2,
                    tq.BusinessDayConvention.ModifiedFollowing,
                    tq.DayCounterConvention.Actual360,
                    1.0)

ois_builder = tq.OisHelper(tq.Currency.EUR,
                        2,
                        2,
                        "1Y",
                        "1Y",
                        tq.BusinessDayConvention.ModifiedFollowing,
                        1.0,
                        tq.DayCounterConvention.Actual360,
                        tq.DayCounterConvention.Actual360)

In [23]:
#bootstrap della estr
generators = modifica_stringhe(ir_eur_curve_estr['type'])
maturities = list(modifica_stringhe(ir_eur_curve_estr['tenor']))
data = [["EUR", "DISCOUNT", "", "EUR:ESTR"]]
attributes = ["CCY", "USAGE"]

In [24]:
curve_assignment = CurveAssignment(data, attributes)

In [25]:
instance = {"CCY": "EUR", "USAGE": "DISCOUNT"}
print(curve_assignment.get_curve_name(instance))

EUR:ESTR
